In [1]:
# Six Small Algorithm Projects
# 1.     Trees
# 1.5.   Drawing

In [2]:
import tkinter as tk

In [3]:
#Create a Binary Node class
class BinaryNode:
    indent = "  "
    node_radius = 10  # Radius of a node’s circle
    x_spacing = 20    # Horizontal distance between neighboring subtrees
    y_spacing = 20    # Vertical distance between parent and child subtrees    
    
    def __init__(self, value):
        self.value = value
        self.left_child = None
        self.right_child = None
        
        # Initialize drawing parameters.
        self.center = (0, 0)
        self.subtree_bounds = (
            self.center[0] - BinaryNode.node_radius, 
            self.center[1] - BinaryNode.node_radius, 
            self.center[0] + BinaryNode.node_radius, 
            self.center[1] + BinaryNode.node_radius)

        
    def add_left(self, node):
        self.left_child = node
        
    def add_right(self, node):
        self.right_child = node
        
    def __str__(self, level=0):
        '''Recursively create a string representation of this node's subtree.
        Display this value indented, followed by the left and right values indented one more level.
        End in a newline.'''

        # Create a string named result that initially holds the
        # current node’s value followed by a new line.
        result = level * BinaryNode.indent + f'{self.value}:\n'

        # If the node has any children:
        if self.left_child or self.right_child:
            # If the node has a left child, add None or the child's value.
            if self.left_child:
                result += self.left_child.__str__(level + 1)
            else:
                result += (level + 1) * BinaryNode.indent + "None\n"

            # If the node has a right child, add None or the child's value.
            if self.right_child:
                result += self.right_child.__str__(level + 1)
            else:
                result += (level + 1) * BinaryNode.indent + "None\n"
        
        #If the node has no children:
        return result

    def find_node(self, target):
        '''Recursively search the subtree rooted at this node for the target value.
        If found, return the node containing the target value, otherwise return None.'''
        if self.value == target:
            return self
        
        if self.left_child != None:
            left_result = self.left_child.find_node(target)
            if left_result != None:
                return left_result
        
        if self.right_child != None:
            right_result = self.right_child.find_node(target)
            if right_result != None:
                return right_result
        
        return None

    def traverse_preorder(self):
        # Visit the node, then recursively visit the node’s children
        result = [self.value]
        if self.left_child:
            result += self.left_child.traverse_preorder()
        if self.right_child:
            result += self.right_child.traverse_preorder()
        return result


    def traverse_postorder(self):
        # Visit the node’s children and then the node
        result = []
        if self.left_child:
            result += self.left_child.traverse_postorder()
        if self.right_child:
            result += self.right_child.traverse_postorder()
        result.append(self.value)
        return result

    def traverse_breadth_first(self):
        # Visit all of the nodes at each level of the tree in left-to-right order
        result = []
        queue = [self]
        while queue:
            node = queue.pop(0)
            result.append(node.value)
            if node.left_child:
                queue.append(node.left_child)
            if node.right_child:
                queue.append(node.right_child)
        return result
    
    def arrange_subtree(self, xmin, ymin):
        '''Position the node's subtree.'''
        # Calculate cy, the Y coordinate for this node.
        # This doesn't depend on the children.
        # ...
        # cy = 100
        cy = ymin + BinaryNode.node_radius

        # If the node has no children, just place it here and return.
        if (self.left_child == None) and (self.right_child == None):
            cx = xmin + BinaryNode.node_radius
            self.center = (cx, cy)
            xmax = xmin + 2 * BinaryNode.node_radius
            ymax = ymin + 2 * BinaryNode.node_radius
            self.subtree_bounds = (xmin, ymin, xmax, ymax)
            return

        # Set child_xmin and child_ymin to the
        # start position for child subtrees.
        child_xmin = xmin
        child_ymin = ymin + 2 * BinaryNode.node_radius + BinaryNode.y_spacing

        # Position the child subtrees.
        if self.left_child != None:
            # Arrange the left child subtree and update
            # child_xmin to allow room for its subtree.
            self.left_child.arrange_subtree(child_xmin, child_ymin)
            child_xmin = self.left_child.subtree_bounds[2]

            # If we also have a right child,
            # add space between their subtrees.
            if self.right_child != None:
                child_xmin += BinaryNode.x_spacing

        if self.right_child != None:
            # Arrange the right child subtree.
            self.right_child.arrange_subtree(child_xmin, child_ymin)

        # Arrange this node depending on the number of children.
        if (self.left_child != None) and (self.right_child != None):
            # Two children. Center this node over the child nodes.
            # Use the child subtree bounds to set our subtree bounds.
            cx = (self.left_child.center[0] + self.right_child.center[0]) / 2
            self.center = (cx, cy)
            xmax = self.right_child.subtree_bounds[2]
            ymax = max(self.left_child.subtree_bounds[3], self.right_child.subtree_bounds[3])
            self.subtree_bounds = (xmin, ymin, xmax, ymax)
        elif self.left_child != None:
            # We have only a left child.
            cx = self.left_child.center[0]
            self.center = (cx, cy)
            xmax = self.left_child.subtree_bounds[2]
            ymax = self.left_child.subtree_bounds[3]
            self.subtree_bounds = (xmin, ymin, xmax, ymax)
        else:
            # We have only a right child.
            cx = self.right_child.center[0]
            self.center = (cx, cy)
            xmax = self.right_child.subtree_bounds[2]
            ymax = self.right_child.subtree_bounds[3]
            self.subtree_bounds = (xmin, ymin, xmax, ymax)

    def draw_subtree_links(self, canvas):
        '''Draw the subtree's links.'''
        if self.left_child != None:
            self.left_child.draw_subtree_links(canvas)
            canvas.create_line(
                self.center[0], self.center[1],
                self.left_child.center[0], self.left_child.center[1],
                fill='red')
        if self.right_child != None:
            self.right_child.draw_subtree_links(canvas)
            canvas.create_line(
                self.center[0], self.center[1],
                self.right_child.center[0], self.right_child.center[1],
                fill='black')

        # Outline the subtree for debugging.
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')

    def draw_subtree_nodes(self, canvas):
        '''Draw the subtree's nodes.'''
        # Draw the node.
        x0 = self.center[0] - BinaryNode.node_radius
        y0 = self.center[1] - BinaryNode.node_radius
        x1 = self.center[0] + BinaryNode.node_radius
        y1 = self.center[1] + BinaryNode.node_radius
        canvas.create_oval(x0, y0, x1, y1, fill='white', outline='green')
        canvas.create_text(self.center, text=self.value, fill='red')

        # Draw the descendants' nodes.
        if self.left_child != None:
            self.left_child.draw_subtree_nodes(canvas)
        if self.right_child != None:
            self.right_child.draw_subtree_nodes(canvas)

    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree.
        self.arrange_subtree(xmin, ymin)

        # Draw the links.
        self.draw_subtree_links(canvas)

        # Draw the nodes.
        self.draw_subtree_nodes(canvas)

In [4]:
# Build a test tree.
#         A
#        / \
#       /   \
#      /     \
#     B       C
#    / \     / \
#   D   E   F   G
#      / \     /
#     H   I   J
#            / \
#           K   L
a = BinaryNode('A')
b = BinaryNode('B')
c = BinaryNode('C')
d = BinaryNode('D')
e = BinaryNode('E')
f = BinaryNode('F')
g = BinaryNode('G')
h = BinaryNode('H')
i = BinaryNode('I')
j = BinaryNode('J')
k = BinaryNode('K')
l = BinaryNode('L')

a.add_left(b)
a.add_right(c)
b.add_left(d)
b.add_right(e)
c.add_left(f)
c.add_right(g)
e.add_left(h)
e.add_right(i)
g.add_left(j)
j.add_left(k)
j.add_right(l)

In [5]:
def kill_callback():
    '''A callback to destroy the tkinter window.'''
    window.destroy()

In [6]:
# Make the tkinter window.
window = tk.Tk()
window.title('binary_node5')
window.protocol('WM_DELETE_WINDOW', kill_callback)
window.geometry('260x220')

canvas = tk.Canvas(window, bg='white', borderwidth=2, relief=tk.SUNKEN)
canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Draw the tree.
a.arrange_and_draw_subtree(canvas, 10, 10)

window.focus_force()
window.mainloop()